# 🔥 DWsolutions


Znasz już nasz DWClub'owy cykl `DS master class`? Pewnie obiło się o uszy, a mam nadzieję, że udało Ci się też praktycznie zaangażować. `DS master class` to format, który polega nauce budowania prototypu konkretnego rozwiązania np. prognozowanie cen mieszkań. Samodzielnie działamy i w ramach konkursu na Kaggle wyłaniamy najlepszy prototyp :) 

`DWsolutions` to cykl, którego celem także jest zbudowanie jak najlepszego prototypu - przebijamy wynik z konkretnego master class, ale WSPÓLNIE. Tak, dokładnie, łączymy siły i wspólnie jako członkowie DW Club próbujemy "pobić" poprzedni model angażując się w twórczą pracę i pomysły. 

### 🤔 Po co to robimy? 

Przede wszystkim naszym celem jest wspólna nauka i jak zwykle dobra zabawa. Natomiast warto wiedzieć, że na tym nie kończy się `DWsolutions`, które jest 2-etapowe. Po części dot. usprawniania modelu **będziemy wdrażać model**... Tak, każdy będzie mógł zobaczyć wynik predykcji modelu w swoim Excelu i skorzystać. 


### 💡Zadanie  

Zadaniem jest zrobienie predykcji cen nieruchomości na rynku polskim. Mamy dane historyczne z Warszawy (ogłoszenia sprzedaży nieruchomości) wraz z informacją o nieruchomości.

Warto wiedzieć💡 - w Polsce jest startup SonarHome, który właśnie wycenia wartość mieszkań - tutaj możesz poczytać/posłuchać rozmowy na ten temat  "[wyceń swoje mieszkanie za pomocą Machine Learning](https://biznesmysli.pl/wycen-swoje-mieszkanie-z-pomoca-machine-learning/)"

Naszą metryką sukcesu będzie: [MAE](https://en.wikipedia.org/wiki/Mean_absolute_error).

Pamiętaj, pracujesz razem z innymi uczestnikami na wspólny zespołowy wynik, który ma dać lepsze rozwiązanie niż to, od którego startujemy (widoczny na Kaggle). To połączenie rywalizacji (gramy o lepszy wynik na Kaggle od tego, który został już osiągnięty podczas Master Class). Trzeba komunikować się i współdziałać z innymi, bo przebicie wyniku nie będzie wcale łatwe! 
Uwierz: w zespole siła 💪


### 🌎 Osiągnięty wynik na Kaggle

Konkurs jest zakończony, ale nadal dostępny na Kaggle - link do [konkursu](https://www.kaggle.com/t/8f76d26546f64ba0ae5e46bf7602859b).

### 📃 Rejestracja

Jeśli jeszcze nie masz konta na Kaggle, to proszę [załóż](https://www.kaggle.com/). Podczas rejestracji jest wymagane potwierdzenie SMS. Zawsze na moich poprzednich warsztatach była 1-2 osoby, które pod żadnym pozorem nie chciały podać swojego numeru telefonu. Jeśli nie chcesz, to OK... w takim razie możesz wykonać to zadanie tylko w notebooku (jako zadanie domowe). Uczestnictwo w konkursie jest opcjonalne. 

Z drugiej strony, Kaggle to jest największa społeczność uczenia maszynowego na świecie. Dlatego jeśli chcesz zająć się tym na poważnie, zainspirować się lub od czasu do czasu brać udział w konkursach na Kaggle (np. tych, które w DataWorkshop regularnie organizujemy) - to warto mieć tam konto. Jak jesteś w [DW Club](https://dataworkshop.eu/dw-club), to wiesz ;) 

### 👀 Porównywanie swojego wyniku z wynikami na Kaggle

Aby przesłać swój plik z wynikiem do Kaggle, musisz przejść do konkursu i kliknąć **"Late Submission"**:

W wyświetlonym oknie wybierz **dany plik** (opcjonalnie dodaj opis do tego pliku), a następnie kliknij poniżej **"Make Submission"**:

Przesłany plik będzie miał tylko 2 kolumny: id (identyfikator zapisu) oraz value (prognozowana cena nieruchomości)

Po jego przesłaniu pojawią Ci się wyniki (score), który uzyskałbyś na Public i Private Leaderboard, gdyby konkurs nadal był aktywny. Główne tablice z wynikami liderów konkursu są już "zamrożone", więc tym razem nie ma ograniczeń jeśli chodzi o liczbę przesyłanych plików  w ciągu dnia (w terminologii Kaggle nazywa się to 'submit'- po zakończeniu konkursu ich liczba nie jest już ograniczona). 


### ❗ Dane

W danych jest **60 222** ogłoszenia (wierszy), które zostały podzielone prawie na równe cześci:
- train (13 947 wierszy)
- test (46 275 wierszy)

Twoim zadaniem jest zrobić predykcję dla zbioru testowego.


## 📺 Pamiętaj o webinarium

Zerknij tutaj i zainspiruj się do działania. 

In [1]:
%%html
<iframe style="height:500px;width:100%" src="https://www.youtube.com/embed/3A-jdxfEcc8" frameborder="0" allow="autoplay; encrypted-media" allowfullscreen></iframe>

In [2]:
import pandas as pd
import numpy as np
np.random.seed(0)

from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score

from sklearn.dummy import DummyRegressor
import xgboost as xgb

## 🗂️ Dane
To są prawdziwe dane.

**Uwaga!** Dane można używać tylko w **celach edukacyjnych** (również nie można ich publikować lub dzielić się nimi z innymi)!

In [3]:
train = pd.read_hdf('../input/train_warsaw_property.h5')
test = pd.read_hdf('../input/test_warsaw_property.h5') #zbiór testowy bez odpowiedzi

In [4]:
train.shape[0] +  test.shape[0]

60222

## 🤖 Basic Model 

Jak zwykle zaczynamy od czegoś prostego, aby mieć jakikolwiek wynik na start, który potem można usprawniać. 

In [ ]:
feats = ['id']  ##cokolwiek na początek

model = DummyRegressor()
model.fit(train[ feats ].values, train['price'])
y_pred = model.predict(test[ feats ].values)

## 🌎 Submission
Trzeba przygotować plik, który będzie zawierał: `id` i `price`.

In [ ]:
test['price'] = y_pred
test[ ['id', 'price'] ].to_csv('../output/model_dummy.csv', index=False) 

![](../images/dummy.png)

## 💪 Bardziej złożony model

In [ ]:
if "price" in test:
    del test["price"]#usuwamy ze zbioru testowego kolumnę "price", to nasza zmienna docelowa, którą chcemy prognozować
    
df = pd.concat([train, test])#łączymy ze sobą zbiór testowy i zbiór treningowy,nowe cechy chcemy tworzyć na obu zbiorach jednocześnie

for feat in df.select_dtypes("object"):#dokonujemy zamiany zmiennych kategorialnych na numeryczne przez zastosowanie .factorize() tam gdzie to możliwe
    try:
        df["{}_cat".format(feat)] = df[feat].factorize()[0]
    except:
        print("can not factorize {}".format(feat))

## 😎 Sprawdzamy ile mamy cech numerycznych

In [ ]:
num_feats = df.select_dtypes("number").columns
len(num_feats)

Mamy już 20 cech numerycznych! Czas aby wytrenować bardziej złożony model.


## 💪 Przygotowujemy `X` i `y`

In [ ]:
train = df[ df['price'].notnull() ]#rozdzielamy zbiór na zbiór treningowy
test = df[ df['price'].isnull() ]#i testowy

print(train.shape, test.shape) #sprawdzamy ile kolumn i wierszy mamy w każdym zbiorze

X_train, y_train = train[feats].values, train["price"].values
X_test = test[feats].values

## 🤖 Trenujemy model

In [ ]:
model = xgb.XGBRegressor(max_depth=7, n_estimators=100, random_state=0)

scores = cross_val_score(model, X_train, y_train, cv=5, scoring="neg_mean_absolute_error")
np.mean(scores), np.std(scores)

## Współpraca 💪 i komunikacja 💬

 
 👉 *współpraca, komunikacja, kreatywne i krytyczne myślenie*  - to wartości i umiejętności, które warto pielęgnować. 
 
W DWClub czeka na Ciebie twórcza atmosfera, fajna zabawa i dobre wyniki. Zaangażuj się i korzystaj z tego maksymalnie. Współpracując z innymi uczestnikami nad osiągnięciem lepszego wyniku, doświadczasz pracy zespołowej i nabywasz tę ważną umiejętność, której przecież nie da się kształtować w pojedynkę.🤷‍♂️

Na potrzeby komunikacji i wsparcia DW Club mamy dedykowany kanał na Slacku - [#dwclub_members](https://dataworkshopclub.slack.com/archives/C02C2KDAS91) - wszystkie inicjatywy DW Club 🥰. 

Dla cyklu DWsolutions są dedykowane kanały: 

[#dwsolutions](https://dataworkshopclub.slack.com/archives/C03GKEA31NW), w którym możesz zadać pytanie lub zgłosić problem związany z przerabianym zadaniem - śmiało umieść kod z błędem i opisz w czym potrzebujesz pomocy 🤝

[#dwsolutions_ideas](https://dataworkshopclub.slack.com/archives/C03HBPNHYTA), w którym dziel się z innymi pomysłami jakie przychodzą Ci do głowy na temat rozwiązania problemu zawartego w projekcie. Opisz, co można zrobić, jeśli potrafisz to zrobić to wklej gotowy kod, którego użyłeś/aś, tak by ktoś inny mógł z niego skorzystać. Podglądaj też propozycje innych uczestników - rozwijaj ich pomysły i weź udział w dyskusji. W tym kanale dbajmy o porządek: kontynuujemy rozpoczęty wątek, a nowy pomysł umieszczamy w osobnym, by inni mogli go komentować i dalej rozwijać 🧠

[#dwsolutions_done](https://dataworkshopclub.slack.com/archives/C03GFBZ89F0) to kanał, w którym należy umieścić rozwiązanie zadań z materiału w ramach ``DWsolutions``. Możesz wykonać plan minimum i otrzymać imienny certyfikat, ale możesz też pochwalić się tym, co więcej udało Ci się uzyskać pracując nad poprawą wyniku. Każdy jest na innym poziomie wtajemniczenia i to jest właśnie wartościowe, jeśli umiemy dzielić się i uczyć od siebie nawzajem 💪



### 🧠 Dlaczego warto intensywnie działać w kanale [#dwsolutions_ideas](https://dataworkshopclub.slack.com/archives/C03HBPNHYTA)? 

Uczenie maszynowe opierają sie zawsze na współpracy wielu osób, to także połączenie wiedzy technicznej i biznesowej. I to jest właśnie fajne, ale także wcale nie takie oczywiste. W ramach DWClub chcemy pomóc budować Ci różne kompetencje, zarówno twarde, jak i miękkie. Właśnie dlatego poza pisaniem kodu i trenowaniem modeli o cichu, zachęcamy Cię, aby szczególnie w w ramach `DWsolutions` podziałać z innymi i wzmocnić swoje umiejętności 4K: komunikacja, kooperacja, kreatywne myślenie i krytyczne myślenie. Poza tym wymiana zdań z innymi i wspólny brainstorming 🧠🧠🧠🌩️ pomaga wpadać na lepsze pomysły. Działamy? 💪

# Twój feedback na koniec dnia - ok. 20-60 sek ;) 👇👇👇

Uzupełnij proszę ankitę poniżej (ok. ok. 20-60 sek). Twoja zwrotna informacja wpływa na to, w którą stronę `DW Club` będzie rozwijał się. Uważnie analizujemy i razem z Tobą zbudujemy najlepszy produkt! 💪

In [ ]:
%%html
<div data-tf-widget="txPZJ3tL" data-tf-iframe-props="title=DWsoultions - 1 day - feedback" data-tf-medium="snippet" style="width:100%;height:400px;"></div><script src="//embed.typeform.com/next/embed.js"></script>